# Space X contracts

#### Load Python tools

In [1]:
%load_ext lab_black

In [15]:
import pandas as pd
import altair as alt
import altair_grid as altgrid
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.float_format", lambda x: "%.2f" % x)

#### Google Sheets config

In [17]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "credentials.json", scope
)
gc = gspread.authorize(credentials)
spreadsheet_key = "1rCoN2RIBBT4SZXPe5OT1GLI4-q0UB5H20TV71KGGOoQ"
book = gc.open_by_key(spreadsheet_key)

---

In [5]:
duns = "120406462"

In [6]:
src = pd.read_csv(
    "data/raw/Contracts_PrimeAwardSummaries_2022-04-27_H17M36S48_1.csv",
    dtype={"recipient_parent_duns": str},
)

In [7]:
src.total_obligated_amount.sum()

10695208656.380001

In [8]:
df = (
    src[src["recipient_parent_duns"] == duns][
        [
            "award_id_piid",
            "recipient_parent_name",
            "award_description",
            "awarding_agency_name",
            "awarding_sub_agency_name",
            "type_of_contract_pricing",
            "total_obligated_amount",
            "potential_total_value_of_award",
            "award_base_action_date",
            "award_base_action_date_fiscal_year",
        ]
    ]
    .sort_values("total_obligated_amount", ascending=False)
    .copy()
)

In [9]:
df.total_obligated_amount.sum().round()

7327613208.0

In [10]:
df.head()

,award_id_piid,recipient_parent_name,award_description,awarding_agency_name,awarding_sub_agency_name,type_of_contract_pricing,total_obligated_amount,potential_total_value_of_award,award_base_action_date,award_base_action_date_fiscal_year
66,NNJ09GA04B,SPACE EXPLORATION TECHNOLOGIES CORP.,ISS COMMERCIAL RESUPPLY SERVICES.,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,3011395263.58,3100000000.00,2008-12-22,2009
21,NNK14MA74C,SPACE EXPLORATION TECHNOLOGIES CORP.,IGF::OT::IGF THE COMMERCIAL CREW PROGRAM (CCP...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,1240950821.00,3511811270.00,2014-09-16,2014
64,80MSFC20C0034,SPACE EXPLORATION TECHNOLOGIES CORP.,"WORK REQUIRED FOR THE DESIGN, DEVELOPMENT, MAN...",NATIONAL AERONAUTICS AND SPACE ADMINISTRATION ...,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,FIRM FIXED PRICE,530667416.24,3031455921.24,2020-05-13,2020
63,FA881119C0004,SPACE EXPLORATION TECHNOLOGIES CORP.,EVOLVED EXPENDABLE LAUNCH VEHICLE 1A-6 LAUNCH ...,DEPARTMENT OF DEFENSE (DOD),DEPT OF THE AIR FORCE,FIRM FIXED PRICE,310871630.30,310871630.30,2019-02-21,2019
61,FA881118C0001,SPACE EXPLORATION TECHNOLOGIES CORP.,IGF::OT::IGF EVOLVED EXPENDABLE LAUNCH VEHICLE...,DEPARTMENT OF DEFENSE (DOD),DEPT OF THE AIR FORCE,FIRM FIXED PRICE,243895249.55,437551474.55,2018-03-14,2018


---

#### Companies

In [22]:
companies = (
    df.groupby(["recipient_parent_name"])
    .agg({"total_obligated_amount": sum, "potential_total_value_of_award": sum})
    .reset_index()
    .sort_values("total_obligated_amount", ascending=False)
)
companies["share"] = (
    companies["total_obligated_amount"] / companies["total_obligated_amount"].sum()
) * 100
companies

,recipient_parent_name,total_obligated_amount,potential_total_value_of_award,share
0,SPACE EXPLORATION TECHNOLOGIES CORP.,7327613207.73,26805081908.62,100.00


#### Agencies

In [23]:
agencies = (
    df.groupby(["awarding_sub_agency_name"])
    .agg({"total_obligated_amount": sum, "potential_total_value_of_award": sum})
    .reset_index()
    .sort_values("total_obligated_amount", ascending=False)
)
agencies["share"] = (
    agencies["total_obligated_amount"] / companies["total_obligated_amount"].sum()
) * 100
agencies = agencies[agencies["total_obligated_amount"] > 0]
agencies

,awarding_sub_agency_name,total_obligated_amount,potential_total_value_of_award,share
5,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,5401176411.57,20220295876.98,73.71
0,DEPT OF THE AIR FORCE,1805904519.74,6258504665.22,24.65
6,SPACE DEVELOPMENT AGENCY,119783656.00,305532746.00,1.63
7,WASHINGTON HEADQUARTERS SERVICES (WHS),498636.42,498636.42,0.01
1,DEPT OF THE ARMY,114984.00,114984.00,0.00
4,MISSILE DEFENSE AGENCY (MDA),100000.00,100000.00,0.00
2,DEPT OF THE NAVY,35000.00,35000.00,0.00


#### Years

In [24]:
years = (
    df.groupby(["award_base_action_date_fiscal_year"])
    .agg({"total_obligated_amount": sum, "potential_total_value_of_award": sum})
    .reset_index()
    .sort_values("award_base_action_date_fiscal_year", ascending=False)
)
years["share"] = (
    years["total_obligated_amount"] / companies["total_obligated_amount"].sum()
) * 100
years = years[years["total_obligated_amount"] > 0]
years

,award_base_action_date_fiscal_year,total_obligated_amount,potential_total_value_of_award,share
13,2022,17405000.00,317400000.00,0.24
12,2021,420560293.37,729070241.24,5.74
11,2020,890216573.02,17304909898.26,12.15
10,2019,380958994.09,380958994.09,5.20
9,2018,484651933.56,678308158.56,6.61
8,2017,97098381.49,97098381.50,1.33
7,2016,511801356.97,429101356.97,6.98
6,2014,1246875515.00,3511811270.00,17.02
5,2013,257493133.00,247923608.00,3.51
4,2011,294921.00,0.00,0.00


---

## Export

In [25]:
wks_name = "years"
d2g.upload(years, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'years' id:2093114953>

In [26]:
wks_name = "agencies"
d2g.upload(
    agencies, spreadsheet_key, wks_name, credentials=credentials, row_names=False
)

<Worksheet 'agencies' id:1420304259>

In [27]:
wks_name = "totals"
d2g.upload(
    companies, spreadsheet_key, wks_name, credentials=credentials, row_names=False
)

<Worksheet 'totals' id:1208025930>

In [28]:
wks_name = "all_contracts"
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

---

## Charts

In [32]:
agencies

,awarding_sub_agency_name,total_obligated_amount,potential_total_value_of_award,share
5,NATIONAL AERONAUTICS AND SPACE ADMINISTRATION,5401176411.57,20220295876.98,73.71
0,DEPT OF THE AIR FORCE,1805904519.74,6258504665.22,24.65
6,SPACE DEVELOPMENT AGENCY,119783656.00,305532746.00,1.63
7,WASHINGTON HEADQUARTERS SERVICES (WHS),498636.42,498636.42,0.01
1,DEPT OF THE ARMY,114984.00,114984.00,0.00
4,MISSILE DEFENSE AGENCY (MDA),100000.00,100000.00,0.00
2,DEPT OF THE NAVY,35000.00,35000.00,0.00
